# Market Learning
*Leveraging machine learning to help you learn the market and invest your money*

### Objective
The Market Learning project is intended to provide an adaptive, fact-based introduction of the stock market to a user with limited experience looking for help with personal investment decisions.

### Outline
- Data Collection
- Data Cleaning
- Visualization
- Prediction
- Conclusion

In [2]:
import requests
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd


### Data Collection
Collect data from sources


In [3]:
# Trading economics
url_1 = 'https://tradingeconomics.com/united-states/stock-market'
page_1 = requests.get(url_1)
page_1.status_code

403

In [4]:
# Wall street journal
url_2 = 'https://www.wsj.com/market-data/quotes/index/SPX/advanced-chart'
page_2 = requests.get(url_2)
page_2.status_code

403

In [5]:
# Yahoo finance
url_3 = 'https://finance.yahoo.com/quote/%' + '5EDJI/history?period1=694310400&period2=1681084800&interval=1mo&filter=history&frequency=1mo&includeAdjustedClose=true'
page_3 = requests.get(url_3)
page_3.status_code

404

I was unable to webscrape several of the websites I found that contained sufficient historical data. 
Yahoo Finance lets users download .csv files of individual stocks' data, so I manually downloaded the following:

In [7]:
# Make an np array of dataframes so I don't need to memorize each acronym
df_ixic = pd.read_csv('^IXIC.csv')
df_ixic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2724 entries, 0 to 2723
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       2724 non-null   object 
 1   Open       2724 non-null   float64
 2   High       2724 non-null   float64
 3   Low        2724 non-null   float64
 4   Close      2724 non-null   float64
 5   Adj Close  2724 non-null   float64
 6   Volume     2724 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 149.1+ KB


In [8]:
df_index = pd.read_csv('INDEX.csv')
df_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413 entries, 0 to 412
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       413 non-null    object 
 1   Open       413 non-null    float64
 2   High       413 non-null    float64
 3   Low        413 non-null    float64
 4   Close      413 non-null    float64
 5   Adj Close  413 non-null    float64
 6   Volume     413 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 22.7+ KB


In [9]:
url_4 = 'https://www.google.com/finance/quote/.IXIC:INDEXNASDAQ?window=MAX'
page_4 = requests.get(url_4)
page_4.status_code

200

In [ ]:
# Initialize lists of stock ticker info
ticker = []
name = []
market_cap = []
stock_price = []
day_change = []
stock_volume = []
market_time = []
security_type = []
primary_exchange = []
industry = []
dividend_yield = []
PE_ratio = []
other_categories = []
